# Canny Edge Detector with trackbars to change threshold values

## Canny Edge detection algorithm

It is a complex and popular multiple steps edge detection algorithm which was developed by [John F.Canny](https://www2.eecs.berkeley.edu/Faculty/Homepages/canny.html) in 1986. 
The steps involved are:
#### 1.) Image smoothening:
This is done to remove the noise from the image and is implemented using a *circular* 2-D Gaussian function. Say, our input image is represented by *f(x,y)* and *G(x,y)* denotes our Gaussian function. <pre>
                               *G(x,y) = e<sup>-(x<sup>2</sup> + y<sup>2</sup>)/2σ<sup>2</sup>*</pre>
Let *f<sub>s</sub>(x,y)* denote the output smoothed image formed by convolving *f(x,y)* and *G(x,y)*. So, it can be mathematically represented as<pre>                               *f<sub>s</sub>(x,y)* = *G(x,y)* \* *f(x,y)* </pre>
Where \* represents the convolution operation.<pre></pre>
We generally use a 5x5 Gaussian filter for performing the convolution operation which is represented below:
![alt text](http://www.pages.drexel.edu/~nk752/Research/GaussianMask.gif "Gaussian Filter")
____________________________________________________________________

#### 2.) Finding intensity gradient magnitude and direction (angle):
*f<sub>s</sub>(x,y)*,ie, the smoothened image is then filtered using the sobel filter mask (or any other filter like Robetrs, Prewitt etc) in both the horizantal and vertical direction to get *g<sub>x</sub>*
and *g<sub>y</sub>* which are the first derivative in horizontal and vertical directions respectively. These are used to compute gradient magnitude and direction, ie, angle as <pre>
                                *M(x,y) = √(g<sub>x</sub><sup>2</sup> + g<sub>x</sub><sup>2</sup>)*
                                *α(x,y) = tan<sup>-1</sup>(g<sub>x</sub>/g<sub>x</sub>)* </pre>
                                

*M(x,y)* is an array representing the  edge strength or gradient magnitude and *α(x,y)* is an array representing the edge direction. Both of these array are of same size as the input image.<pre>
__Note:__ Gradient direction is always perpendicular to edges.</pre>
_____________________________________________________________________________________

#### 3.) Applying Non-maximum Suppression to *M(x,y)* :
One thing to remember is that 1<sup>st</sup> order derivatives generally produce thicker edges in an image. Since *M(x,y)* is produced using gradients, ie, first-order derivatives therfore we need to make those edges thin. This can be achieved using *non-maximum suppression*. The approach to be followed is to specify a number of different orientations of the gradient vectors which are parallel to the edge (as mentioned in the note).To explain this, let's take a 3x3 region.We can define 4 possible orientations for an edge passing through the center point namely __horizontal, +45<sup>O</sup>, -45<sup>O</sup>, vertical__. Now, we want to categorise each of the values of *α(x,y)* (rmember that α is a an array) into one of the above four categories, for this we'd take the help of the image below.
![alt text](https://i.imgur.com/XnKziTD.png "Gradient Categories")

We can determine the direction of the edge from the direction of gradient vector(or array), ie,*α(x,y)*. Seeing the diagram above, we can see that the regions of same shade of gray belongs to the same category of edge. For example, edge from -22.5<sup>O</sup> to 22.5<sup>O</sup> and -157.5<sup>O</sup> to 157.5<sup>O</sup>belong to horizontal edge region because their gradients are in vertical direction.

Now, after segregating all the gradients into the 4 categories and having their corresponding gradient values, ie, *M(x,y)* we'll go about the whole image and would try to remove the pixels which don't constitute the edge.To do this, we'll take a pixel gradient magnitude say, *M(x1,y1)* and see if it greater than the two of its neighbours along the direction of the gradient (ie, perpendicular to the edge direction). If its gradient magnitude is greater than the neighbouring pixel's then we take itas gradient value as it is, ie, *M(x1,y1)* but if its not, then it gradient value is __suppressed to zero.__ 
This paragraph explains why it is called as **non-maximum suppression**. 
<pre>The image below better explains the supression process.</pre>

![alt text](https://i.imgur.com/OAW0V83.png "Non-max Supression")

The arrow indicates the gradient direction. Point q is on the edge and points p and r lie in the direction of gradient. Now if, gradient intensity of point q is less that the gradient intenstiy of both point p and q, then it is suppressed to zero and if its not, then point q will be considered for the next step, ie, if *M(x<sub>q</sub>,y<sub>q</sub>) < M(x<sub>p</sub>,y<sub>p</sub>) * && *M(x<sub>q</sub>,y<sub>q</sub>) < M(x<sub>r</sub>,y<sub>r</sub>)* then *M(x<sub>q</sub>,y<sub>q</sub>)* is suppressed (made zero) otherwise it is kept as it is and carried to next step.
______________________________________________________________________________________________________

#### 4.) Hysteresis Thresholding:

This is a double thresholding process which helps us in determining the real edges. We have two threshold values *MaxVal* and *MinVal*. Any edge with intensity value more than *MaxVal* are sure-edge and any edge with intensity value less than *MinVal* are non-edge. Now, we need to look upon the edges having intensity values between *MinVal* and *MaxVal*. If these edges are connected to the sure-edges then they are also considered to be the part of the edges otherwise they are discarded.
The figure below explains it more clearly.
![alt text](https://i.imgur.com/U8Qtplq.jpg "Hysteresis Thresholding")

Point X is above the *MaxVal* so it is considered as a sure-edge point. Point W is below the *MinVal* so it is discarded. Now  the remaining two points, ie, Y andZ lie between the threshold values. Since Y is connected to the edge X therefore it is kept as sure-edge but since Z is not connected to any sure-edge therefore it is discarded.

__Note:__ Genral *MaxVal* : *MinVal* ratio is taken as 2:1 or 3:1

So, all this leads us to geeting strong edges in our image.

## Python Implementation

To conduct the Canny Edge Detection algorithm, we will use the [cv2.Canny()](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_canny/py_canny.html#canny) fuction. Instead of just vaguely using the canny function, I have incorporated two trackbars representing the Maximum and Minimum threshold values so that we can better understand the hysteresis thresholding.

In [ ]:
#importing the required libraries
import cv2
import numpy as np

#Function needed as an argument of createTrackbar()
def nothing(x):
    pass

#Forming a window with the name Canny_Edge_Detection
cv2.namedWindow('Canny_Edge_Detection')

#creating two trackbars representing Maximum and Minimum thresholds
cv2.createTrackbar('MaxVal', 'Canny_Edge_Detection',0,255,nothing)
cv2.createTrackbar('MinVal', 'Canny_Edge_Detection',0,255,nothing)

img = cv2.imread('Image path',0)

while(1):
    #getting the trackbar value
    MaxVal =cv2.getTrackbarPos('MaxVal', 'Canny_Edge_Detection')
    MinVal = cv2.getTrackbarPos('MinVal', 'Canny_Edge_Detection')
    #Applying the canny edge detection
    edges = cv2.Canny(img,MinVal,MaxVal)

    cv2.imshow("Canny_Edge_Detection", edges)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
cv2.destroyAllWindows()


# Result Visualisation
Here is a gif showing changes in edges on changing the threshold values

![alt text](https://i.imgur.com/3XPT7ha.gif "Result")

### References:
1.  [OpenCV Python Tutorials](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html)
2. [Canny_edge detection](http://www.pages.drexel.edu/~nk752/Research/cannyTut2.html)
3. [Lecture Slides NCTU](http://caig.cs.nctu.edu.tw/course/CV09/Vision_5Edge_S09.pdf)
4. [Digital Image Processing book](https://www.amazon.in/Digital-Image-Processing-3-Old/dp/8131726959)
5. [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#links)

*You are free to use this work but please leave a to **this repository** :)*